# SESI 15
Nama : Syaima Radestya <br>
Kode Peserta : PYTN-KS10-011 <br>
Program : Introduction to Python for Data Science

# MODEL DEPLOYMENT
***
Data scientist sering kali harus menyampaikan hasil kepada orang lain. Dalam beberapa 
kasus, supervisor/atasan mungkin ingin melihat beberapa angka atau kita harus menuliskan main 
insights dari beberapa pekerjaan. Ini cukup mudah - kita hanya copy dan paste ke email atau 
dokumen Latex. Tetapi bagaimana jika kita ingin mengirim seseorang model sebenarnya yang 
telah kita latih, sehingga mereka dapat mengevaluasi atau menggunakannya? Jika orang yang
dimaksud cukup teknis, maka kita bisa menyimpan model ke disk dan kirim via email, bersama 
dengan beberapa notebook Jupyter. <br> <br>
Tetapi bagaimana jika kita ingin seseorang di bagian marketing mencoba sebuah model, 
atau kita ingin membagikannya dengan teman-teman yang mungkin menganggapnya menarik? 
Dalam kasus ini, kita harus membuat kode kita dapat diakses oleh mereka dengan cara yang 
mereka kenal. <br> <br>
Tiga bagian dari sesi ini adalah: 
1. Greating a simple model that can be deployed to the web, where users can input variables 
to get predictions.
2. Building the components needed by Flask microframework to create a web app.
3. Deploying the final web app using the Heroku platform.

In [63]:
# import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [64]:
# open data
df = pd.read_csv("train.csv", index_col=0)
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [65]:
# hapus kolom nama
df.drop(['Name', 'Ticket', 'Embarked', 'Cabin'], axis=1, inplace=True)

In [66]:
# imputasi
median = df.Age.median()
df.Age.fillna(median, inplace=True)

In [67]:
df.Sex = df.Sex.map({'female':0, 'male':1})

In [68]:
# modelling
# split dahulu baru scale

X = df.drop('Survived', axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=46)

In [69]:
# scaling (hanya boleh di itung menggunakan data training)
# data testing (transformasi yang didapatkam dari data training)

scaler = StandardScaler()
scaler.fit(X_train)

# transformasi X nya
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [70]:
# modelling
model = LinearSVC()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

c:\Users\User\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [71]:
# Simpan Objeknya
import pickle

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [72]:
with open("svm.pkl", "wb") as f:
    pickle.dump(model, f)

In [73]:
# buat pipeline
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    StandardScaler(),
    LinearSVC()
)

pipe.fit(X_train, y_train)

c:\Users\User\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC())])

In [74]:
with open("pipeline.pkl", "wb") as f:
    pickle.dump(pipe, f)

In [75]:
y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.89      0.85       104
           1       0.83      0.72      0.77        75

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179



In [76]:
with open("pipeline.pkl", "rb") as f:
    model = pickle.load(f)

In [77]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare
PassengerId,,,,,,
625,3,1,21.0,0,0,16.1000
698,3,0,28.0,0,0,7.7333
472,3,1,38.0,0,0,8.6625
66,3,1,28.0,1,1,15.2458
180,3,1,36.0,0,0,0.0000


In [78]:
X_test.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [79]:
columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
new_data = [1, 0, 20, 0, 2, 500]

In [80]:
from sqlalchemy import column


df_data = pd.DataFrame([new_data], columns=columns)
df_data

,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,20,0,2,500


In [81]:
model.predict(df_data)

array([1], dtype=int64)